In [2]:
import pandas as pd

def save_fastest_laps_to_excel(df, output_file):
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        df.to_excel(writer, index=False, sheet_name="Dati")
        
        workbook = writer.book
        worksheet = writer.sheets["Dati"]
        for col in worksheet.iter_cols(min_row=1, max_row=1, min_col=1, max_col=len(df.columns)):
            for cell in col:
                cell.style = 'Pandas'

from openpyxl import load_workbook
def append_to_excel(filename, df, sheet_name="Dati"):
    try:
        # Carica il file esistente
        workbook = load_workbook(filename)
        if sheet_name in workbook.sheetnames:
            sheet = workbook[sheet_name]
            start_row = sheet.max_row + 2  # Calcola la riga di partenza per il nuovo DataFrame
        else:
            sheet = workbook.create_sheet(sheet_name)
            start_row = 1
    except FileNotFoundError:
        # Se il file non esiste, crealo
        workbook = None
        start_row = 1

    # Scrivi i dati sul file
    with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
        #print(writer.book)
        #writer.sheets = {ws.title: ws for ws in workbook.worksheets}
        df.to_excel(writer, index=False, header=start_row == 1, startrow=start_row - 1, sheet_name=sheet_name)

In [4]:
import fastf1 as ff1 
Q = ff1.get_session(2024, 'Austin', 4)
Q.load()

req         WARNING 	DEFAULT CACHE ENABLED! (491.04 MB) C:\Users\giacomotommi\AppData\Local\Temp\fastf1
core           INFO 	Loading data for United States Grand Prix - Qualifying [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '55', '16', '81', '63', '10', '14', '20', '11', '22', '27', '31', '18', '30', '23', '43', '77', '44', '24

In [6]:
drivers = pd.unique(Q.laps['Driver'])       # ottengo la lista dei piloti in ordine di arrivo
Q1, Q2, Q3 = Q.laps.split_qualifying_sessions()    # divido le sessioni di qualifica
drivers

array(['NOR', 'VER', 'SAI', 'LEC', 'PIA', 'RUS', 'GAS', 'ALO', 'MAG',
       'PER', 'TSU', 'HUL', 'OCO', 'STR', 'LAW', 'ALB', 'COL', 'BOT',
       'HAM', 'ZHO'], dtype=object)

Ottengo i valori di Perez e Lawson che in altro modo non riuscirei ad ottenere avendo fallito i loro giri lanciati nelle varie sessioni

list_fastest_laps = list()
for drv in drivers[15:]:
    drvs_fastest_lap = Q1.pick_drivers(drv).pick_fastest()
    list_fastest_laps.append(drvs_fastest_lap)
df = pd.DataFrame(Laps(list_fastest_laps).sort_values(by='LapTime').reset_index(drop=True))